In [1]:
# Import statements
import string
import numpy as np
import pandas as pd
import scipy
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [2]:
df = pd.read_csv('pos-eng-5000.data.csv')
v = LabelEncoder()
center = np.array(df['a4'])
print(center)
class_code = v.fit_transform(df[['class']])
class_inverted = v.inverse_transform((class_code))
class_map = {}

for i in range(len(class_code)):
    if class_inverted[i] not in class_map:
        class_map[class_inverted[i]] = [class_code[i], 1]
    else:
        class_map[class_inverted[i]][1] += 1
lpo = float('inf')
lpo_class = ""
for key in class_map:
    if class_map.get(key)[1] < lpo:
        lpo = class_map.get(key)[1]
        lpo_class = class_map.get(key)[0]
        

['The' 'cafeteria' 'remains' ... 'frenzy' 'PERIOD' 'I']


/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [3]:
word_class_dict = {}
for i in range(len(center)):
    if center[i] not in word_class_dict:
        word_class_dict[center[i]] = class_code[i]
print(word_class_dict)    

{'The': 8, 'cafeteria': 16, 'remains': 37, 'closed': 12, 'PERIOD': 21, 'Some': 8, 'analysts': 19, 'argued': 33, 'that': 11, 'there': 9, 'wo': 15, 'nSQt': 25, 'be': 32, 'a': 8, 'flurry': 16, 'of': 11, 'takeovers': 19, 'because': 11, 'the': 8, 'industry': 16, 'SQs': 22, 'continuing': 12, 'capacity-expansion': 12, 'program': 16, 'is': 37, 'eating': 34, 'up': 28, 'available': 12, 'cash': 16, 'A': 8, 'number': 16, 'developments': 19, 'were': 33, 'negatively': 25, 'interpreted': 35, 'by': 11, 'traders': 19, 'place': 16, 'smelt': 35, 'some': 8, 'kind': 16, 'hair': 16, 'lotion': 16, 'these': 8, 'pimplike': 12, 'characters': 19, 'use': 36, 'MrPERIOD': 17, 'OSQBrien': 17, 'will': 15, 'elaborate': 32, 'on': 11, 'SIA': 17, 'position': 16, 'in': 11, 'testimony': 16, 'before': 11, 'House': 17, 'Telecommunications': 18, 'and': 2, 'Finance': 17, 'Subcommittee': 17, 'this': 8, 'week': 16, 'COMMA': 5, 'spokesman': 16, 'said': 33, 'ailing': 34, 'company': 16, 'which': 38, 'has': 37, 'reported': 35, 'net'

In [35]:
new_df = pd.read_csv('pos-eng-5000.data.csv')
everything = df[['a2', 'a3', 'a4', 'a5', 'a6', 'a7']].to_numpy()
grand_list = []
for row in everything: 
    local_list = []
    for i in row:
        if i in word_class_dict:
            local_list.append(word_class_dict.get(i))
        else:
            local_list.append(lpo_class)
    grand_list.append(local_list)


X = scipy.sparse.csr_matrix(grand_list)
Y = scipy.sparse.csr_matrix(class_code)

In [36]:
# Define a function to take as input training and testing vectors and labels
# Allow this to be extensible to let multiple classifiers be used here
def buildClassifiers(clf, X_train, X_test, y_train, y_test):

    clf.fit(X_train.todense(), y_train)
    y_pred = clf.predict(X_test.todense())

    f1 = f1_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    accuracy = accuracy_score(y_test, y_pred)
    return f1, precision, recall, accuracy


In [37]:
names = ['Naive_Bayes', 'Decision_Tree']
classifiers =   [GaussianNB(), 
                DecisionTreeClassifier(random_state=0)]
for name, clf in zip(names, classifiers):

    print('Now classifying', name)

    # Fold the data 5 times
    kf = KFold(n_splits = 5)
    foldCounter = 0
    aList, bList, cList, dList= list(), list(), list(), list()
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train = []
        y_test = []
        for i in train_index:
            y_train.append(Y[0, i])
        for i in test_index:
            y_test.append(Y[0, i])
        f1, precision, recall, accuracy= buildClassifiers(clf, X_train, X_test, y_train, y_test)
        aList.append(f1)
        bList.append(precision)
        cList.append(recall)
        dList.append(accuracy)

    print("\tAverage Macro F1 for {}:\t\t".format(name), np.mean(aList))
    print("\tAverage Macro Precision for {}:\t".format(name), np.mean(bList))
    print("\tAverage Macro Recall for {}:\t\t".format(name), np.mean(cList))
    print("\tAverage Macro Accuracy for {}:\t\t".format(name), np.mean(dList))


Now classifying Naive_Bayes


/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/s

	Average Macro F1 for Naive_Bayes:		 0.6699465521631961
	Average Macro Precision for Naive_Bayes:	 0.6932628762612766
	Average Macro Recall for Naive_Bayes:		 0.6850814494009813
	Average Macro Accuracy for Naive_Bayes:		 0.8055999999999999
Now classifying Decision_Tree


/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


	Average Macro F1 for Decision_Tree:		 0.8351008976248766
	Average Macro Precision for Decision_Tree:	 0.841596774196591
	Average Macro Recall for Decision_Tree:		 0.8517182168733861
	Average Macro Accuracy for Decision_Tree:		 0.95


/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-pack